In [18]:
import numpy as np
import pandas as pd
import re
from convokit import Corpus, download
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, f1_score
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
import gender_guesser.detector as gender

In [19]:
# corpus = Corpus(filename=download('supreme-corpus'))
all_utts = pd.read_csv('data/utts.csv')

In [20]:
# All cases
cases = pd.read_json(path_or_buf='data/cases.jsonl', lines=True)

# Cases with clear winners
df = cases.loc[cases.loc[:, 'win_side'].isin([1, 0])]

# Roberts court cases with clear winners
roberts = df.loc[df.loc[:, 'court'] == 'Roberts Court', :]

# All utterances
#all_utts = corpus.get_utterances_dataframe()

# Roberts court case utterances
roberts_ids = roberts.loc[:, 'id'].unique()
utts = all_utts.loc[all_utts.loc[:, 'meta.case_id'].isin(roberts_ids)]

# Roberts court cases with clearn winners and utterance data
    # Unique case IDs from utts indicate the number of Roberts court cases
    # the corpus has utterance data for
subset_ids = utts.loc[:, 'meta.case_id'].unique()
    # Use these ids to subset the roberts data frame
    # (exclude cases without utterance data)
roberts = roberts.loc[roberts.loc[:, 'id'].isin(subset_ids)]
print(len(roberts), 'cases')
petitioner_wins = roberts.loc[:, 'win_side'].mean()
print(round(petitioner_wins * 100, 2),
      'percent of cases were decided favorably for the petitioner')

1024 cases
65.33 percent of cases were decided favorably for the petitioner


In [21]:
# Utterances per case
print('Min:', utts.groupby(['meta.case_id']).size().min()) 
print('Max:', utts.groupby(['meta.case_id']).size().max())
print('Mean', utts.groupby(['meta.case_id']).size().mean())

Min: 1
Max: 1235
Mean 237.3955078125


In [22]:
# Drop single-utterance cases
utt_counts = pd.DataFrame(utts.groupby(['meta.case_id']).size())
utt_counts = utt_counts.reset_index()
utt_counts = utt_counts.rename(columns={0: 'utt_counts'})
utts = pd.merge(utts, utt_counts, how = 'left')
utts = utts.loc[utts.loc[:, 'utt_counts'] != 1, :]
print('1 case dropped')
print('New min:', utts.groupby(['meta.case_id']).size().min()) 
print(len(utts.groupby(['meta.case_id'])), 'cases')

# Store utts for use in other notebooks
%store utts

1 case dropped
New min: 66
1023 cases
Stored 'utts' (DataFrame)


Index(['id', 'timestamp', 'text', 'speaker', 'reply_to', 'conversation_id',
       'meta.case_id', 'meta.start_times', 'meta.stop_times',
       'meta.speaker_type', 'meta.side', 'meta.timestamp', 'vectors',
       'utt_counts'],
      dtype='object')

In [29]:
# Drop single-sided cases

# Case IDs with 0 or 100% petitioner_advocate_utt_share
%store -r advocates
single_sided = (advocates.loc[
    ((advocates.loc[:, 'petitioner_advocate_utt_share'] == 0.0) |
     (advocates.loc[:, 'petitioner_advocate_utt_share'] == 1.0)), :])
single_sided = single_sided.reset_index()

# advocates has one row per case ID --> case_ids are already unique
iffy_ids = single_sided.loc[:, 'meta.case_id']
utts = utts.loc[~utts.loc[:, 'meta.case_id'].isin(iffy_ids)]
print(len(utts.groupby(['meta.case_id'])), 'cases')

1004 cases


In [30]:
# Compute justice utterance share
utts['justice'] = utts.loc[:, 'meta.speaker_type'] == 'J'
df = utts.groupby('meta.case_id').agg({'justice': ['sum', 'count']})
df.columns = [ 'justice_utts', 'tot_utts']
df['justice_utt_share'] = df.loc[:, 'justice_utts'] / df.loc[:, 'tot_utts']
justices = df.loc[:, ['justice_utt_share']]
print('Min:', justices.loc[:, 'justice_utt_share'].min())
print('Max:', justices.loc[:, 'justice_utt_share'].max())
print('Mean:', justices.loc[:, 'justice_utt_share'].mean())

Min: 0.48
Max: 0.6521739130434783
Mean: 0.5152917822677271


In [31]:
# Compute petitioner advocate utterance share
advocates = utts.loc[utts.loc[:, 'meta.speaker_type'] == 'A', :].copy()
advocates['petitioner_advocate'] = advocates.loc[:, 'meta.side'] == 1
advocates = advocates.groupby('meta.case_id').agg({'petitioner_advocate': ['sum', 'count']})
advocates.columns = [ 'petitioner_advocate_utts', 'total_advocate_utts']
advocates['petitioner_advocate_utt_share'] = advocates.loc[:, 'petitioner_advocate_utts'] / advocates.loc[:, 'total_advocate_utts']
advocates = advocates.loc[:, [ 'petitioner_advocate_utt_share']]
print('Min', advocates.loc[:, 'petitioner_advocate_utt_share'].min())
print('Max', advocates.loc[:, 'petitioner_advocate_utt_share'].max())
print('Mean', advocates.loc[:, 'petitioner_advocate_utt_share'].mean())
%store advocates

Min 0.14130434782608695
Max 0.967741935483871
Mean 0.46024237757454767
Stored 'advocates' (DataFrame)


In [32]:
# Compute female utterance share
d = gender.Detector()

def guess_gender(name):
    name = re.sub('j__', '', name)
    name = re.sub('_\S*', '', name)
    name = name.capitalize()
    guess = d.get_gender(name, 'usa')
    return guess

# Guess gender
gendr = utts.loc[:, ['meta.case_id', 'speaker' ]].copy()
gendr.loc[:, 'gender'] = gendr.loc[:, 'speaker'].map(guess_gender)
print('Before subsetting:')
print(gendr.loc[:, 'gender'].unique())
print(gendr.groupby('gender').agg({'gender': ['count']}).droplevel)

# Restrict to confident guesses
gendr = gendr.loc[gendr.loc[:, 'gender'].isin(['female', 'mostly_female', 
                                               'mostly_male', 'male'])]
print('\nAfter subsetting:')
print(gendr.loc[:, 'gender'].unique())
print(gendr.groupby('gender').agg({'gender': ['count']}).droplevel)

# Compute female_utt_share 
    # = (N female + N mostly_female) / 
    # (N female + N mostly_female + N male + N mostly_male)
gendr.loc[:, 'gender_num'] = gendr.loc[:, 'gender'].isin(['female', 
                                                          'mostly_female'])
gendr = gendr.groupby('meta.case_id').agg({'gender_num': ['sum', 'count']})
gendr.columns = ['female_utts', 'total_utts']
gendr.loc[:, 'female_utt_share'] = (gendr.loc[:, 'female_utts'] /
                                    gendr.loc[:, 'total_utts'])
gendr = gendr.loc[:, ['female_utt_share']]

Before subsetting:
['male' 'female' 'andy' 'mostly_male' 'unknown' 'mostly_female']
<bound method NDFrame.droplevel of                gender
                count
gender               
andy            28747
female          41059
male           156479
mostly_female    1856
mostly_male      3099
unknown          7680>

After subsetting:
['male' 'female' 'mostly_male' 'mostly_female']
<bound method NDFrame.droplevel of                gender
                count
gender               
female          41059
male           156479
mostly_female    1856
mostly_male      3099>


In [33]:
# Combine utterance shares
utt_shares = pd.merge(justices, advocates, how = 'left', left_index = True, right_index = True)
utt_shares = pd.merge(utt_shares, gendr, how = 'left', left_index = True, right_index = True)
utt_shares.head()

,justice_utt_share,petitioner_advocate_utt_share,female_utt_share
meta.case_id,,,
2005_04-1034,0.513353,0.603659,0.066390
2005_04-10566,0.525333,0.533708,0.140000
2005_04-1067,0.534954,0.483660,0.275081
2005_04-1084,0.526611,0.514793,0.358025
2005_04-1131,0.509434,0.584615,0.403361


In [34]:
#Code to create conversations/case feature
utts['text'].replace('\d+', '', regex=True, inplace = True) 
convo_case = pd.DataFrame(utts.groupby('meta.case_id')['conversation_id'].nunique())
convo_case.rename(columns = {'conversation_id':'convo_count'}, inplace = True)
print(convo_case.loc[:,'convo_count'].describe())

"""PROPORTION OF UTTERANCES BY JUSTICES APPOINTED BY REPUBLICAN PRESIDENTS
Does not consider length of utterance, which also may be of interest"""
just_dict = {'j__john_g_roberts_jr':0, 'j__david_h_souter':0,
       'j__anthony_m_kennedy':0, 'j__ruth_bader_ginsburg':1,
       'j__john_paul_stevens':0, 'j__samuel_a_alito_jr':0,
       'j__antonin_scalia':0, 'j__stephen_g_breyer':1,
       'j__sandra_day_oconnor':0, 'j__clarence_thomas': 0 ,
       'j__sonia_sotomayor':1, 'j__elena_kagan':1, 'j__neil_gorsuch': 0,
       'j__brett_m_kavanaugh':0} #based on being appointed by cons/liberal president
j_utts = utts.copy()
j_utts = j_utts.loc[utts.loc[:,'justice'] == True]
j_utts.loc[:,'cons_just']= j_utts.loc[:,'speaker'].map(just_dict)
cons_just = j_utts.groupby(['meta.case_id']).agg({'cons_just':'mean'}) 
print(cons_just.loc[:,'cons_just'].describe())

count    1004.000000
mean        1.016932
std         0.150480
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         4.000000
Name: convo_count, dtype: float64
count    1004.000000
mean        0.434504
std         0.155229
min         0.045455
25%         0.307692
50%         0.444994
75%         0.553393
max         0.915789
Name: cons_just, dtype: float64


In [35]:
#PROPORTION OF REPUBLICAN APPOINTED JUSTICES ON THE COURT AT THE TIME OF THE DECISION
roberts.loc[roberts.loc[:,'id'] == '2019_17-1268','decided_date'] = '2020-06-19'
roberts.loc[:,'decided_date'] = pd.to_datetime(roberts.loc[:,'decided_date'])
roberts.loc[roberts.loc[:,'decided_date'].between('2005-10-29','2009-08-07'), 'prop_cons'] = 7/9
roberts.loc[roberts.loc[:,'decided_date'].between('2009-08-08','2010-08-06'), 'prop_cons'] = 6/9
roberts.loc[roberts.loc[:,'decided_date'].between('2010-08-07','2016-02-12'), 'prop_cons'] = 5/9
roberts.loc[roberts.loc[:,'decided_date'].between('2016-02-13','2017-04-09'), 'prop_cons'] = 4/8
roberts.loc[roberts.loc[:,'decided_date'].between('2017-04-10','2020-07-10'), 'prop_cons'] = 5/9

prop_cons = roberts.loc[:,['id','prop_cons']].set_index('id')
print(roberts.loc[:,'prop_cons'].describe())

ValueError: time data "2020-06-19" doesn't match format "%b %d, %Y", at position 427. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [8]:
group_utts = utts.groupby('meta.case_id')['text'].apply(' '.join)
df = pd.merge(group_utts,roberts[['id','win_side']], how = 'left', 
              left_on = 'meta.case_id', right_on = 'id')
X_train, X_test, y_train, y_test = train_test_split(
    df.loc[:,df.columns != 'win_side'],df['win_side'], test_size = 0.2, random_state = 0)

In [11]:
#Initialize Vectorizer and vectorize train data
count_vect = CountVectorizer(ngram_range = (1,1),min_df = 4,
            stop_words = 'english', token_pattern = r'\b[a-zA-Z]{3,}\b') 
count_df = count_vect.fit_transform(X_train['text'])
count_array = count_df.toarray()
count_df = pd.DataFrame(count_array,columns = count_vect.get_feature_names(), index = X_train['id'])
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(count_df).toarray()
X_train_tfidf = pd.DataFrame(X_train_tfidf, columns = count_vect.get_feature_names(), index = X_train['id'])

X_train_tfidf = pd.merge(X_train_tfidf,convo_case, how = 'left', left_index = True, right_index = True)
X_train_tfidf = pd.merge(X_train_tfidf,utt_shares, how = 'left', left_index = True, right_index = True)
X_train_tfidf = pd.merge(X_train_tfidf,cons_just, how = 'left', left_index = True, right_index = True)
X_train_tfidf = pd.merge(X_train_tfidf,prop_cons, how = 'left', left_index = True, right_index = True)
#MERGING IN ADDITIONAL FEATURES HERE
X_train_tfidf

,aaa,aba,abandon,abandoned,abandoning,abandonment,abandons,abate,abated,abatement,...,zip,zipes,zone,zones,zoning,convo_count,justice_utt_share,petitioner_advocate_utt_share,cons_just,prop_cons
id,,,,,,,,,,,,,,,,,,,,,
2015_15-290,0.000000,0.000000,0.007648,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,1,0.532934,0.423077,0.662921,0.500000
2007_07-343,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,1,0.515982,0.584906,0.221239,0.777778
2009_09-497,0.006649,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,1,0.512727,0.425373,0.432624,0.666667
2005_04-1131,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.011321,0.006974,0.0,0.0,1,0.509434,0.584615,0.377778,0.777778
2009_09-338,0.000000,0.008387,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,1,0.519573,0.518519,0.294521,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016_16-6219,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,1,0.515982,0.462264,0.663717,0.555556
2007_06-984,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,1,0.516279,0.341346,0.337838,0.777778
2013_12-79,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,1,0.502674,0.354839,0.414894,0.555556


In [12]:
#Vectorize test data
count_test = count_vect.transform(X_test['text'])
count_test_array = count_test.toarray()
count_test_df = pd.DataFrame(count_test_array, columns = count_vect.get_feature_names(),index = X_test['id'])

X_test_tfidf = tfidf_transformer.transform(count_test_df).toarray()
X_test_tfidf = pd.DataFrame(X_test_tfidf, columns = count_vect.get_feature_names(), index = X_test['id'])
X_test_tfidf = pd.merge(X_test_tfidf,convo_case, how = 'left', left_index = True, right_index = True)
X_test_tfidf = pd.merge(X_test_tfidf,utt_shares, how = 'left', left_index = True, right_index = True)
X_test_tfidf = pd.merge(X_test_tfidf,cons_just, how = 'left', left_index = True, right_index = True)
X_test_tfidf = pd.merge(X_test_tfidf,prop_cons, how = 'left', left_index = True, right_index = True)
X_test_tfidf

,aaa,aba,abandon,abandoned,abandoning,abandonment,abandons,abate,abated,abatement,...,zip,zipes,zone,zones,zoning,convo_count,justice_utt_share,petitioner_advocate_utt_share,cons_just,prop_cons
id,,,,,,,,,,,,,,,,,,,,,
2016_15-649,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,1,0.512048,0.222222,0.729412,0.500000
2005_04-1544,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,1,0.502110,0.483051,0.378151,0.777778
2006_05-1074,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,1,0.513514,0.537037,0.324561,0.777778
2013_12-158,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.004829,0.0,0.0,1,0.520958,0.475000,0.528736,0.555556
2009_08-1553,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,1,0.503425,0.296552,0.632653,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015_14-419,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,1,0.524887,0.466667,0.448276,0.500000
2009_08-1529,0.0,0.0,0.0,0.005098,0.021389,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,1,0.505814,0.235294,0.494253,0.666667
2009_09-475,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,1,0.506024,0.463415,0.365079,0.666667


In [14]:
logit = LogisticRegression()
logit = logit.fit(X = X_train_tfidf, y = y_train)
logit.predict(X_test_tfidf)
mean_acc = logit.score(X_test_tfidf, y_test)
print(mean_acc)

0.6731707317073171


In [26]:
np.mean(y_test)

0.6292682926829268

In [17]:
#CONSTRUCTED FEATURE ONLY MODEL
X_train_feat = X_train_tfidf.loc[:,['convo_count',
       'justice_utt_share', 'petitioner_advocate_utt_share', 'cons_just','prop_cons']]
X_test_feat = X_test_tfidf.loc[:,['convo_count',
       'justice_utt_share', 'petitioner_advocate_utt_share', 'cons_just','prop_cons']]
logit2 = LogisticRegression()
logit2 = logit2.fit(X = X_train_feat, y = y_train)
logit2.predict(X_test_feat)
mean_acc2 = logit2.score(X_test_feat, y_test)
print(mean_acc2)

0.6292682926829268


In [23]:
mlp = MLPClassifier(hidden_layer_sizes=(100,50,25), activation='relu', 
                    solver='adam', max_iter=500, random_state = 0)
mlp.fit(X_train_feat,y_train)
predict_test = mlp.predict(X_test_feat)
accuracy_score(y_test,predict_test)

0.6585365853658537

In [24]:
print(confusion_matrix(y_test,predict_test)) #lots of false negatives
print(classification_report(y_test,predict_test))

[[ 11  65]
 [  5 124]]
              precision    recall  f1-score   support

         0.0       0.69      0.14      0.24        76
         1.0       0.66      0.96      0.78       129

    accuracy                           0.66       205
   macro avg       0.67      0.55      0.51       205
weighted avg       0.67      0.66      0.58       205

